# Choose Dataset

In [42]:
d_class      = 'depression' # / 'control'

dataset_type = 'time'
month        = 3

print(d_class, dataset_type, month, 'month')

##############################################

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

depression time 3 month


## Load raw text of each tweet with user ID

In [ ]:
pd.options.mode.chained_assignment = None

# d1_text contain text of each tweets
d1_text = pd.read_csv(f'data_{dataset_type}/{d_class}/raw_{month}m.csv', sep='|', header = 0, lineterminator='\n', dtype= str, index_col=False)
d1_text["user_id"] = d1_text['user_id'].apply(pd.to_numeric)
d1_text.head()

In [21]:
print(d1_text.shape)

(470356, 2)


In [22]:
len(d1_text['user_id'].unique())

3000

In [23]:
d1_text['text'].to_csv(f'data_{dataset_type}/{d_class}/text_only_{month}m.csv', index=False, sep="|", line_terminator='\n', header = False)
d1_text['user_id'].to_csv(f'data_{dataset_type}/{d_class}/user_id_{month}m.csv', index=False, sep="|", line_terminator='\n', header = False)

## STOP HERE !!!!
Convert text_only.csv to .txt

In [43]:
# Preprocessor can REMOVE these things
# URL	            p.OPT.URL
# Mention	        p.OPT.MENTION
# Hashtag	        p.OPT.HASHTAG
# Reserved Words	p.OPT.RESERVED for twitters ex RT
# Emoji	            p.OPT.EMOJI
# Smiley	        p.OPT.SMILEY

import preprocessor as p

input_file_name = f'data_{dataset_type}/{d_class}/text_only_{month}m.txt'
p.clean_file(input_file_name, p.OPT.URL, p.OPT.MENTION, p.OPT.HASHTAG, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.SMILEY)
print("Success !!!")

# number of line is still equal to the original d1_text

Saved the cleaned tweets to:data_time/depression/29052022_191051063258_DJdriI30nvaN_text_only_3m.txt
Success !!!


# Rename to pclean.txt !!!

In [47]:
import re
import string
from bs4 import BeautifulSoup
from html import unescape

In [48]:
## NEW 05/2022 Ver.

def preprocess(row):
    
    row = re.sub("\[\]\((.*?)\)", "", row) # remove these things that comes with Reddit [](/trixiesmug)
    row = re.sub("\\r\\\w+",'', row) # remove these things \r\books
    row = re.sub("r/\w+",'', row) # remove these things r/books
    
    # Unescape HTML tags
    row = BeautifulSoup(unescape(row), 'lxml').text
    
    # Remove all punctuations except . ! ? $ % & '
    row = re.sub(r"""[()#[\]#*+\-/:;<=>@[\]^_`{|}~"\\]""", "", row)
    # Remove Duplicated punctuations 
    row = re.sub(r"""(?<=[^!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~][!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~])[!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~ ]+(?<! )""", "", row)
    row = re.sub(r'!{2,}', r'!', row)
    row = re.sub(r'\.{2,}', r'.', row)
    row = re.sub(r'\&{2,}', r'&', row)
    row = re.sub(r'\%{2,}', r'%', row)
    row = re.sub(r'\?{2,}', r'?', row)
    row = re.sub(r'\${2,}', r'$', row)

    # Lowercase
    row = ' '.join([w.lower() for w in row.split()])
    
    # Remove extra spaces
    row = ' '.join([w for w in row.split()])

    return row

In [49]:
input_file = f'data_{dataset_type}/{d_class}/pclean_{month}m_0529.txt'
output_file = f'data_{dataset_type}/{d_class}/finclean_{month}m_0529.txt'

with open(input_file) as f, open(output_file, 'w') as o:
    lines = f.readlines()
    # print(len(lines))
    for line in lines:
        # print(line)
        cleaned = preprocess(line)
        
        o.write(cleaned + "\n")